3.3. CIFAR-10 画像認識

CIFAR-10 は、カナダのトロント大学が提供する、10種類の画像を分類するためのデータセットです。このデータセットは、32x32ピクセルのカラー画像 60,000枚で構成されています。そのうち、50,000枚が訓練用、10,000枚がテスト用になっています。画像は、飛行機、自動車、鳥、猫、鹿、犬、カエル、馬、船、トラックの10種類です。

In [15]:
import os
import keras

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Flatten, Dense, Dropout

from keras.datasets import cifar10

from keras.optimizers import RMSprop

from keras.callbacks import TensorBoard, ModelCheckpoint


In [16]:
# LeNet configuration

def network(input_shape, num_classes):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  return model

In [17]:
class CIFAR10Dataset():
  """
  CIFAR-10 dataset.
  attributes:
    - image_shape: image shape
    - num_classes: number of classes
  """
  def __init__(self):
    self.image_shape = (32, 32, 3)
    self.num_classes = 10

  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    x_train /= 255
    x_test /= 255

    y_train = keras.utils.to_categorical(y_train, self.num_classes)
    y_test = keras.utils.to_categorical(y_test, self.num_classes)

    return x_train, y_train, x_test, y_test
  
  def preprocess(slef, data, label_data=False):
    if label_data:
      data = keras.utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype('float32')
      data /= 255
      shape = (data.shape[0],) + self.image_shape
      return data

In [18]:
class Trainer():
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    self.verbose = 1
    self.log_dir = os.path.join(os.getcwd(), 'logdir')
    
    
    self.model_file_name = 'model_file.hdf5'

  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)

    self._target.fit(
      x_train, y_train,
      batch_size=batch_size, epochs=epochs,
      validation_split=validation_split,
      callbacks=[
        TensorBoard(log_dir=self.log_dir),
        ModelCheckpoint(os.path.join(self.log_dir, self.model_file_name), save_best_only=True)
      ],
      verbose=self.verbose)

In [19]:
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

model = network(dataset.image_shape, dataset.num_classes)
trainer = Trainer(model, loss='categorical_crossentropy', optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
313/313 [==============================] - 2s 6ms/step - loss: 1.7606 - accuracy: 0.3611 - val_loss: 1.5824 - val_accuracy: 0.4140
Epoch 2/12
313/313 [==============================] - 1s 5ms/step - loss: 1.3881 - accuracy: 0.5060 - val_loss: 1.2723 - val_accuracy: 0.5449
Epoch 3/12
313/313 [==============================] - 2s 5ms/step - loss: 1.2322 - accuracy: 0.5622 - val_loss: 1.2537 - val_accuracy: 0.5637
Epoch 4/12
313/313 [==============================] - 2s 5ms/step - loss: 1.1372 - accuracy: 0.5978 - val_loss: 1.0958 - val_accuracy: 0.6258
Epoch 5/12
313/313 [==============================] - 2s 5ms/step - loss: 1.0639 - accuracy: 0.6263 - val_loss: 1.0067 - val_accuracy: 0.6512
Epoch 6/12
313/313 [==============================] - 1s 5ms/step - loss: 1.0051 - accuracy: 0.6468 - val_loss: 0.9594 - val_accuracy: 0.6634
Epoch 7/12
313/313 [==============================] - 2s 5ms/step - loss: 0.9580 - accuracy: 0.6628 - val_loss: 0.9594 - val_accuracy: 0.6684
Epoch 

参考書と同じコードだと思うが、結果が大違いだ！

In [20]:
# check tensorboard
# !tensorboard --logdir=logdir

**ネットワークをもっと深くすることによる改善**

In [21]:
# LeNet configuration

def network(input_shape, num_classes):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  return model

In [22]:
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

model = network(dataset.image_shape, dataset.num_classes)
trainer = Trainer(model, loss='categorical_crossentropy', optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12


2024-02-07 15:57:31.613924: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_4/dropout_9/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


313/313 [==============================] - 2s 6ms/step - loss: 1.8431 - accuracy: 0.3327 - val_loss: 1.6507 - val_accuracy: 0.4204
Epoch 2/12
313/313 [==============================] - 2s 5ms/step - loss: 1.4255 - accuracy: 0.4902 - val_loss: 1.2419 - val_accuracy: 0.5583
Epoch 3/12
313/313 [==============================] - 2s 5ms/step - loss: 1.2176 - accuracy: 0.5648 - val_loss: 1.1780 - val_accuracy: 0.5745
Epoch 4/12
313/313 [==============================] - 2s 5ms/step - loss: 1.0982 - accuracy: 0.6108 - val_loss: 0.9344 - val_accuracy: 0.6708
Epoch 5/12
313/313 [==============================] - 2s 5ms/step - loss: 1.0054 - accuracy: 0.6457 - val_loss: 0.8630 - val_accuracy: 0.6944
Epoch 6/12
313/313 [==============================] - 2s 5ms/step - loss: 0.9356 - accuracy: 0.6709 - val_loss: 0.8825 - val_accuracy: 0.6901
Epoch 7/12
313/313 [==============================] - 2s 5ms/step - loss: 0.8719 - accuracy: 0.6941 - val_loss: 0.8475 - val_accuracy: 0.7016
Epoch 8/12
313/31